In [228]:
import numpy as np 
import healpy as hp
import matplotlib.pyplot as plt
from glob import glob

In [231]:
nside = 256
lmax = 3*nside-1
ell_input, TT, TE, EE, BB, PP = np.loadtxt('cl.txt', unpack=True)
# input_cls = np.array([TT, EE, np.zeros_like(EE), TE]) #no BB
input_cls = np.array([TT, EE, BB, TE]) 
input_cls /= ell_input*(ell_input+1)/2/np.pi 
for c in input_cls: c[0] = 0 
input_cls = input_cls[:,:lmax+1]

/tmp/ipykernel_6854/2534342784.py:6: RuntimeWarning: invalid value encountered in divide
  input_cls /= ell_input*(ell_input+1)/2/np.pi


In [232]:
np.random.seed(0)
input_map = hp.synfast(input_cls, nside, new=True)

In [235]:
hp.write_map('map.fits', input_map)

setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]


In [229]:
mapdir = 'test/out/schedule0*'
cov = np.zeros((17, 12*256**2))
invcov = np.zeros((17, 12*256**2))
nw = np.zeros((17, 12*256**2))
hits = np.zeros((17, 12*256**2))
rcond = np.zeros((17, 12*256**2))
for i, schedule in enumerate(sorted(glob(mapdir))):
    # print(schedule)
    cov[i] = hp.read_map(f'{schedule}/filterbin_cov.fits')
    invcov[i] = hp.read_map(f'{schedule}/filterbin_invcov.fits')
    nw[i] = hp.read_map(f'{schedule}/filterbin_noiseweighted_filtered_map.fits')
    m[i] = hp.read_map(f'{schedule}/filterbin_filtered_map.fits')
    hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
    rcond[i] = hp.read_map(f'{schedule}/filterbin_rcond.fits')